No need to rererun these scripts, they are only here to explain how we shoose the datasets

In [12]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta
from io import BytesIO

# =============================================================================
# 1. CONFIGURATION DU BRUTE FORCE
# =============================================================================
CITY = "berlin"
COUNTRY = "germany/be"
SAVE_DIR = "../data"
MIN_FILL_RATE = 0.70  # Votre seuil mathématique strict

# =============================================================================
# 2. GÉNÉRATEUR TEMPOREL (LES 2 DERNIÈRES ANNÉES)
# =============================================================================
def generate_last_two_years_dates():
    """
    Génère une liste de toutes les dates (format YYYY-MM-DD) des 2 dernières années,
    classées de la plus récente à la plus ancienne.
    """
    end_date = datetime.now()
    start_date = end_date - timedelta(days=2 * 365) # Il y a 2 ans
    
    # Utilisation de Pandas pour générer la série temporelle
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # Conversion en string et inversion (on veut tester hier avant de tester l'an dernier)
    dates_str = date_range.strftime('%Y-%m-%d').tolist()[::-1]
    
    print(f"🗓️ {len(dates_str)} dates générées (de {dates_str[0]} à {dates_str[-1]}).")
    return dates_str

# =============================================================================
# 3. FONCTION DE VALIDATION (Strictement la vôtre)
# =============================================================================
def is_dataset_valid(df, date_str):
    core_features = ['price', 'amenities', 'beds', 'bedrooms']
    
    if 'bathrooms_text' in df.columns:
        bath_col = 'bathrooms_text'
    elif 'bathrooms' in df.columns:
        bath_col = 'bathrooms'
    else:
        bath_col = None

    completeness_scores = {}
    for feat in core_features:
        completeness_scores[feat] = df[feat].notnull().mean() if feat in df.columns else 0.0
            
    completeness_scores['bathrooms'] = df[bath_col].notnull().mean() if bath_col else 0.0

    is_valid = True
    print(f"  📊 Évaluation {date_str} (Seuil min: {MIN_FILL_RATE*100}%) :")
    
    for feat, fill_rate in completeness_scores.items():
        if fill_rate < MIN_FILL_RATE:
            print(f"     ❌ ÉCHEC : '{feat}' = {fill_rate*100:.1f}%")
            is_valid = False
        else:
            print(f"     ✅ PASS  : '{feat}' = {fill_rate*100:.1f}%")
            
    return is_valid

# =============================================================================
# 4. PIPELINE DE BRUTE FORCE OPTIMISÉ (AVEC REQUÊTES HEAD)
# =============================================================================
def brute_force_temporal_berlin():
    if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)
        
    dates_to_test = generate_last_two_years_dates()
    print(f"\n--- DÉMARRAGE DU SCAN QUOTIDIEN SUR LES SERVEURS POUR {CITY.upper()} ---")
    
    winning_date = None
    df_valid = None
    hits_found = 0
    
    # Session requests pour optimiser les connexions TCP (Best practice réseau)
    session = requests.Session()
    session.headers.update({'User-Agent': 'Mozilla/5.0'})
    
    for date_str in dates_to_test:
        url = f"http://data.insideairbnb.com/{COUNTRY}/{CITY}/{date_str}/data/listings.csv.gz"
        
        try:
            # OPTIMISATION : Requête HEAD d'abord (ultra-rapide, ne télécharge rien)
            head_response = session.head(url, timeout=5)
            
            if head_response.status_code == 200:
                hits_found += 1
                print(f"\n🎯 [BINGO] Fichier détecté sur le serveur à la date : {date_str}")
                print("   📥 Téléchargement en cours (GET)...")
                
                # Maintenant on télécharge vraiment le fichier
                get_response = session.get(url, stream=True, timeout=20)
                content = BytesIO(get_response.content)
                df_temp = pd.read_csv(content, compression='gzip', low_memory=False)
                
                if is_dataset_valid(df_temp, date_str):
                    winning_date = date_str
                    df_valid = df_temp
                    print(f"\n🏆 VICTOIRE ABSOLUE ! La date {winning_date} cachée valide vos critères.")
                    break
                else:
                    print("   ⏭️ Fichier existant mais recalé par vos critères de qualité. On continue...")
                    
        except requests.exceptions.Timeout:
            # On ignore silencieusement les timeouts sur les requêtes HEAD
            pass 
        except Exception as e:
            print(f"   🚨 Erreur inattendue pour {date_str} : {e}")

    if winning_date and df_valid is not None:
        save_path = f"{SAVE_DIR}/{CITY}_listings_raw_{winning_date}.csv.gz"
        df_valid.to_csv(save_path, index=False, compression='gzip')
        print(f"\n💾 Dataset parfait sauvegardé sous : {save_path}")
        return save_path, df_valid
    else:
        print(f"\n💀 ÉCHEC TOTAL : {hits_found} fichiers trouvés sur 730 jours testés, mais AUCUN ne dépasse 70% de complétude.")
        print("   Conclusion scientifique irrévocable : Les données de Berlin sur cette période sont intrinsèquement incomplètes.")
        return None, None

# Exécution du script
save_path, df_berlin = brute_force_temporal_berlin()

🗓️ 731 dates générées (de 2026-02-21 à 2024-02-22).

--- DÉMARRAGE DU SCAN QUOTIDIEN SUR LES SERVEURS POUR BERLIN ---

🎯 [BINGO] Fichier détecté sur le serveur à la date : 2025-11-24
   📥 Téléchargement en cours (GET)...
  📊 Évaluation 2025-11-24 (Seuil min: 70.0%) :
     ❌ ÉCHEC : 'price' = 64.2%
     ✅ PASS  : 'amenities' = 100.0%
     ❌ ÉCHEC : 'beds' = 64.2%
     ✅ PASS  : 'bedrooms' = 86.0%
     ✅ PASS  : 'bathrooms' = 99.9%
   ⏭️ Fichier existant mais recalé par vos critères de qualité. On continue...

🎯 [BINGO] Fichier détecté sur le serveur à la date : 2025-09-23
   📥 Téléchargement en cours (GET)...
  📊 Évaluation 2025-09-23 (Seuil min: 70.0%) :
     ❌ ÉCHEC : 'price' = 64.9%
     ✅ PASS  : 'amenities' = 100.0%
     ❌ ÉCHEC : 'beds' = 65.0%
     ✅ PASS  : 'bedrooms' = 85.8%
     ✅ PASS  : 'bathrooms' = 99.9%
   ⏭️ Fichier existant mais recalé par vos critères de qualité. On continue...

🎯 [BINGO] Fichier détecté sur le serveur à la date : 2025-07-22
   📥 Téléchargement en cour

On ne garde pas Berlin, on perds trop de donnée. On ne peut pas supposer que les variables manquantes sont iid ce qui détruirait la construction en quantile, on cherche donc une autre ville.

On cherche parmi les villes d'europes ayant une grande proportion d'observations

In [ ]:
import os
import requests
import re
import pandas as pd
from io import BytesIO

# =============================================================================
# 1. CONFIGURATION
# =============================================================================
SAVE_DIR = "../data"
MIN_FILL_RATE = 0.70  # seuil strict garantissant la représentativité
TARGET_DOMAIN_COUNT = 3 

# Liste élargie pour maximiser les chances de trouver 3 excellents datasets
CANDIDATE_CITIES = [
    ("italy/lazio", "rome"),
    ("the-netherlands/north-holland", "amsterdam"),
    ("united-kingdom/england", "london"),
    ("spain/comunidad-de-madrid", "madrid"),
    ("austria/vienna", "vienna"),
    ("ireland/leinster", "dublin"),
    ("portugal/lisbon", "lisbon"),
    ("greece/attica", "athens"),
    ("belgium/bru", "brussels")
]

# =============================================================================
# 2. FONCTIONS DE DÉCOUVERTE ET VALIDATION (Identiques)
# =============================================================================
def get_latest_date_for_city(country, city):
    portal_url = "http://insideairbnb.com/get-the-data/"
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(portal_url, headers=headers, timeout=10)
        pattern = fr"https?://data\.insideairbnb\.com/[^/]+/[^/]+/{city}/([^/]+)/data/listings\.csv\.gz"
        dates_found = re.findall(pattern, response.text)
        if dates_found:
            return sorted(list(set(dates_found)), reverse=True)[0] 
    except Exception:
        pass
    return "2024-03-20" # Fallback

def is_dataset_valid(df, city, date_str):
    core_features = ['price', 'amenities', 'beds', 'bedrooms']
    
    if 'bathrooms_text' in df.columns:
        bath_col = 'bathrooms_text'
    elif 'bathrooms' in df.columns:
        bath_col = 'bathrooms'
    else:
        bath_col = None

    completeness_scores = {}
    for feat in core_features:
        completeness_scores[feat] = df[feat].notnull().mean() if feat in df.columns else 0.0
    completeness_scores['bathrooms'] = df[bath_col].notnull().mean() if bath_col else 0.0

    is_valid = True
    print(f"  📊 Scores pour {city.upper()} :")
    for feat, fill_rate in completeness_scores.items():
        if fill_rate < MIN_FILL_RATE:
            print(f"     ❌ ÉCHEC : '{feat}' = {fill_rate*100:.1f}%")
            is_valid = False
        else:
            print(f"     ✅ PASS  : '{feat}' = {fill_rate*100:.1f}%")
            
    return is_valid

# =============================================================================
# 3. PIPELINE MULTI-DOMAINES
# =============================================================================
def find_top_k_valid_domains(k=3):
    if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)
        
    print(f"--- DÉMARRAGE DU SCANNER : RECHERCHE DU TOP {k} DOMAINES ---")
    
    valid_domains = []
    
    for country, city in CANDIDATE_CITIES:
        # Condition d'arrêt : on a atteint le nombre de villes souhaité
        if len(valid_domains) >= k:
            break
            
        print(f"\n🌍 Évaluation du domaine : {city.upper()} ({len(valid_domains)}/{k} trouvés)")
        latest_date = get_latest_date_for_city(country, city)
        url = f"http://data.insideairbnb.com/{country}/{city}/{latest_date}/data/listings.csv.gz"
        
        try:
            response = requests.get(url, stream=True, timeout=15)
            if response.status_code == 200:
                print("   📥 Téléchargement en mémoire...")
                content = BytesIO(response.content)
                df_temp = pd.read_csv(content, compression='gzip', low_memory=False)
                
                if is_dataset_valid(df_temp, city, latest_date):
                    save_path = f"{SAVE_DIR}/{city}_listings_raw_{latest_date}.csv.gz"
                    df_temp.to_csv(save_path, index=False, compression='gzip')
                    
                    print(f"\n🏆 {city.upper()} validé et sauvegardé !")
                    valid_domains.append({'city': city, 'path': save_path, 'dataframe': df_temp})
                else:
                    print(f"   💥 {city.upper()} disqualifié.")
            else:
                print(f"   ⚠️ Fichier introuvable sur le serveur (Code {response.status_code}).")
                
        except Exception as e:
            print(f"   🚨 Erreur lors du traitement de {city}: {e}")

    # Bilan
    print(f"\n========================================================")
    print(f"🏁 SCAN TERMINÉ : {len(valid_domains)} domaines valides récoltés.")
    if len(valid_domains) > 0:
        print("Villes sélectionnées pour la Généralisation de Domaine :")
        for domain in valid_domains:
            print(f" - {domain['city'].capitalize()}")
    print(f"========================================================\n")
    
    return valid_domains

# Lancement
top_domains = find_top_k_valid_domains(TARGET_DOMAIN_COUNT)

--- DÉMARRAGE DU SCANNER : RECHERCHE DU TOP 3 DOMAINES ---

🌍 Évaluation du domaine : ROME (0/3 trouvés)
   📥 Téléchargement en mémoire...
  📊 Scores pour ROME :
     ✅ PASS  : 'price' = 89.1%
     ✅ PASS  : 'amenities' = 100.0%
     ✅ PASS  : 'beds' = 89.3%
     ✅ PASS  : 'bedrooms' = 97.7%
     ✅ PASS  : 'bathrooms' = 99.9%

🏆 ROME validé et sauvegardé !

🌍 Évaluation du domaine : AMSTERDAM (1/3 trouvés)
   📥 Téléchargement en mémoire...
  📊 Scores pour AMSTERDAM :
     ❌ ÉCHEC : 'price' = 56.0%
     ✅ PASS  : 'amenities' = 100.0%
     ❌ ÉCHEC : 'beds' = 56.3%
     ✅ PASS  : 'bedrooms' = 97.1%
     ✅ PASS  : 'bathrooms' = 99.9%
   💥 AMSTERDAM disqualifié.

🌍 Évaluation du domaine : LONDON (1/3 trouvés)
   📥 Téléchargement en mémoire...
  📊 Scores pour LONDON :
     ❌ ÉCHEC : 'price' = 64.0%
     ✅ PASS  : 'amenities' = 100.0%
     ❌ ÉCHEC : 'beds' = 64.0%
     ✅ PASS  : 'bedrooms' = 86.8%
     ✅ PASS  : 'bathrooms' = 99.8%
   💥 LONDON disqualifié.

🌍 Évaluation du domaine : MADRID (1